In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U bitsandbytes
!pip install -U langchain
!pip install -U langchain_community
!pip install modelbit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 945.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 6.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packa

In [2]:
! ls -l /opt/conda/lib/python3.10/site-packages/aiohttp-*

total 40
-rw-rw-r-- 2 root root    5 Nov 27  2023 INSTALLER
-rw-rw-r-- 2 root root  588 Nov 27  2023 LICENSE.txt
-rw-rw-r-- 2 root root 7357 Nov 27  2023 METADATA
-rw-rw-r-- 2 root root 8465 Nov 27  2023 RECORD
-rw-rw-r-- 2 root root    0 Nov 27  2023 REQUESTED
-rw-rw-r-- 2 root root  105 Nov 27  2023 WHEEL
-rw-rw-r-- 2 root root  103 Nov 27  2023 direct_url.json
-rw-rw-r-- 2 root root    8 Nov 27  2023 top_level.txt


In [3]:
! rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info

In [4]:
 #General packages
#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from textwrap import fill
from IPython.display import Markdown, display # for formating Python display folowing markdown language
import warnings
warnings.filterwarnings('ignore') # avoid warning messages importing packages


In [5]:
# Mixtral and LangChain packages (prompt engineering)
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

2024-06-11 08:33:39.637913: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 08:33:39.638040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 08:33:39.789015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Model version of Mixtral
MODEL_NAME = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"

# Quantization is a technique used to reduce the memory and computation requirements 
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the Mixtral 8x7b-instruct-v0.1, 
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language Mixtral 8x7b-instruct-v0.1
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.7 # randomness of the generated tex
generation_config.top_p = 0.95 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [7]:
# HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=pipe)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [8]:
deposition = """
-- Beginning of Page 1 ---
Line 1, 10:32:18: In The
Line 2, 10:32:18: Matter Of:
Line 3, 10:32:18: Smith vs. City Construction Co.
--- End of Page 1 ---

--- Beginning of Page 2 ---
Line 1, 10:32:18:
--- End of Page 2 ---

--- Beginning of Page 3 ---
Line 1, 10:32:18: Court of Common Pleas
Line 2, 10:32:18: County of Newfield
Line 3, 10:32:18: Case No. 2023-CV-019284
--- End of Page 3 ---

--- Beginning of Page 4 ---
Line 1, 10:32:18: Deposition of Jane Doe,
Line 2, 10:32:18: held at: 123 Broad St., Newfield, NY
Line 3, 10:32:18: Before: Maria Gonzalez, Court Reporter
--- End of Page 4 ---

--- Beginning of Page 5 ---
Line 1, 10:32:18: APPEARANCES
Line 2, 10:32:18: COUNSEL FOR PLAINTIFF:
Line 3, 10:32:18: John Appleseed, Esq.
Line 4, 10:32:18: Appleseed & Associates
Line 5, 10:32:18: 456 Orchard Rd., Suite 300
Line 6, 10:32:18: Newfield, NY 10027
Line 7, 10:32:18: (555) 123-4567
--- End of Page 5 ---

--- Beginning of Page 6 ---
Line 1, 10:32:18: COUNSEL FOR DEFENDANT:
Line 2, 10:32:18: Sally Stone, Esq.
Line 3, 10:32:18: Stone Law Group
Line 4, 10:32:18: 789 Quarry Way, Suite 500
Line 5, 10:32:18: Newfield, NY 10031
Line 6, 10:32:18: (555) 987-6543
--- End of Page 6 ---

--- Beginning of Page 7 ---
Line 1, 10:32:18: Page 1
Line 2, 10:32:18: PROCEEDINGS
Line 3, 10:32:18: Whereupon,
Line 4, 10:32:18: Jane Doe, being first duly sworn, was examined and testified as follows:
Line 5, 10:32:18: DIRECT EXAMINATION
Line 6, 10:32:18: BY MR. APPLESEED:
--- End of Page 7 ---

--- Beginning of Page 8 ---
Line 1, 10:32:18: Q. Please state your name for the record.
Line 2, 10:32:18: A. My name is Jane Doe.
Line 3, 10:32:18: Q. Ms. Doe, how are you currently employed?
Line 4, 10:32:18: A. I am a project manager at City Construction Co.
--- End of Page 8 ---

--- Beginning of Page 9 ---
Line 1, 10:32:18: Q. Were you involved in the project at 500 Main Street on January 5th, 2023?
Line 2, 10:32:18: A. Yes, I was the project manager on-site.
Line 3, 10:32:18: Q. Can you describe what happened on that day?
Line 4, 10:32:18: A. There was an accident involving a pedestrian and a company vehicle.
--- End of Page 9 ---

--- Beginning of Page 10 ---
Line 1, 10:32:18: Q. What were the circumstances leading up to the accident?
Line 2, 10:32:18: A. The area was poorly lit, and the pedestrian unexpectedly crossed in front of the vehicle.
--- End of Page 10 ---

--- Beginning of Page 11 ---
Line 1, 10:32:18: Q. What measures were taken to secure the area?
Line 2, 10:32:18: A. We had standard safety protocols in place, but the lighting issue had not been addressed.
--- End of Page 11 ---

--- Beginning of Page 12 ---
Line 1, 10:32:18: Q. Were there any prior complaints about the lighting in that area?
Line 2, 10:32:18: A. Yes, there had been several complaints which were documented in our project logs.
--- End of Page 12 ---

--- Beginning of Page 13 ---
Line 1, 10:32:18: Q. To your knowledge, were these issues addressed before the accident?
Line 2, 10:32:18: A. They were scheduled for review, but no changes had been made by the date of the accident.
--- End of Page 13 ---

--- Beginning of Page 14 ---
Line 1, 10:32:18: CROSS-EXAMINATION
Line 2, 10:32:18: BY MS. STONE:
--- End of Page 14 ---

--- Beginning of Page 15 ---
Line 1, 10:32:18: Q. Ms. Doe, isn't it true that despite the complaints, no urgent action was taken by your company?
Line 2, 10:32:18: A. That is correct, but we followed the procedures as dictated by our safety guidelines.
--- End of Page 15 ---

--- Beginning of Page 16 ---
Line 1, 10:32:18: Q. Are you saying that the safety of pedestrians was not a priority for your project?
Line 2, 10:32:18: A. Safety is always our priority, but unfortunately, the lighting issue was not resolved in time.
--- End of Page 16 ---

--- Beginning of Page 17 ---
Line 1, 10:32:18: Page 2
Line 2, 10:32:18: EXHIBITS
Line 3, 10:32:18: Exhibit 1 - Project Safety Guidelines
Line 4, 10:32:18: Exhibit 2 - Complaint Log Entries
--- End of Page 17 ---
--- Beginning of Page 18 ---
Line 1, 10:32:18: (Whereupon, the deposition was concluded at 11:58:18.)
--- End of Page 18 

"""

In [9]:
template = """[INST] Summarize the following deposition in approximately 50 words. Include the following:
    - Deponent’s name
    - Case details (name, number, court, date, location)
    - General topic of the case
    - Key topics discussed
    - Pivotal statements
    - Significant objections and rulings
    - Crucial discrepancies or admissions
    - Exhibits referenced
    
    [/INST]

"""

In [10]:
def run_prompt(deposition):
    formatted_text = template.format(text=deposition)
    
    
    result = pipe(formatted_text)
    
    # Extracting summary from the result
    output = result[0]['generated_text']
    
    # Slice the important_statements to remove the template section
    start_marker = "[INST]"
    end_marker = "[/INST]"
    start_index = output.find(start_marker)
    end_index = output.rfind(end_marker)
    
       
    summary = output[end_index + 1:]
        
    return summary


In [11]:
summary = run_prompt(deposition)
print(summary)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/INST]

John Doe deposited in case ABC v. XYZ (123456, District Court, NY, 09/01/2022). This employment dispute case involves allegations of wrongful termination. Topics include job duties, performance reviews, and disciplinary actions. Notable points are Doe's admission to accessing company data post-termination and a disputed timeline regarding his dismissal. Objection for relevance was sustained. No exhibits were referenced.
